In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss

In [2]:
# train = pd.read_pickle(FILE.train_agg_v1.value)
train = pd.read_pickle(FILE.train_agg_v1_select.value)
print('train shape is: {}'.format(train.shape))
# test = pd.read_pickle(FILE.test_agg_v1.value)
test = pd.read_pickle(FILE.test_agg_v1_select.value)
print('test shape is: {}'.format(test.shape))

# train = train1.iloc[:,:200].copy()
# train['click'] = train1['click'].copy()
# test = test1.iloc[:,:200].copy()

train shape is: (1001650, 809)
test shape is: (40024, 808)


# down_sampling

In [ ]:

# def down_sample(train,seed=23,seed_shuffle=9,target='click'):
#     df_posi = train[train[target]==1].copy()
#     print(df_posi.shape)
#     df_neg = train[train[target]==0].copy()
#     print(df_neg.shape)
#     df_neg_sample = df_neg.sample(n=len(df_posi),random_state=seed)
#     print('done selecting')
#     df = pd.concat([df_posi,df_neg_sample])
#     df = df.sample(frac=1,random_state=seed_shuffle)
#     print('done sampling')
#     df = df.reset_index(drop=True)
#     print(df.shape)
#     return df
# train_sample = down_sample(train,seed=23,seed_shuffle=9,target='click')
# # del train
# gc.collect()

# LGBM

In [3]:
from sklearn.model_selection import train_test_split
print_to_file = False 
test_run = False 

def get_time(timezone='America/New_York', time_format='%Y-%m-%d %H:%M:%S'):
    from datetime import datetime
    from dateutil import tz

    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(timezone)

    utc = datetime.utcnow()

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    est = utc.astimezone(to_zone)

    return est.strftime(time_format)

import sys, time
class Logger(object):
    def __init__(self, logtofile=True, logfilename='log'):
        self.terminal = sys.stdout
        self.logfile = "{}_{}.log".format(logfilename, int(time.time()))
        self.logtofile = logtofile

    def write(self, message):
        #         self.terminal.write(message)
        if self.logtofile:
            self.log = open(self.logfile, "a")
            self.log.write('[' + get_time() + '] ' + message)
            self.log.close()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass


def divert_printout_to_file():
    sys.stdout = Logger(logfilename='logfile')

if print_to_file:
    divert_printout_to_file()  # note: comment this to use pdb

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, train_df, test_df, holdout, num_folds, submission_file_name, fe_img_name, stratified = False, debug= False, colsample=0.67, max_depth=8, num_leaves=31, min_child_samples=20, subsample=0.7, reg_lambda=0.3, lr=0.04, seed=1001, verbose=100, rounds=None):
    print(train_df.shape, test_df.shape, holdout.shape)
    print('MEAN: train({}) vs holdout({}): '.format(len(train_df), len(holdout)), train_df['click'].mean(), holdout['click'].mean())
    # Divide in training/validation and test data
    if df is not None:
        train_df = df[df['click'].notnull()]
        test_df = df[df['click'].isnull()]
        print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
        del df
        gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
        
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    holdout_final_preds = np.zeros(holdout.shape[0])
    feature_importance_df = pd.DataFrame()
    feature_importance_gain_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['click','instance_id','index','time']]
    train_scores = []
    holdout_scores = []
    scores = []
    diff_val_holdout = []
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['click'])):
#         print('valid index : ',list(valid_idx)[:5])
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['click'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['click'].iloc[valid_idx]
#         print('MEAN: train({}) vs valid({}): '.format(len(train_y), len(valid_y)), np.mean(train_y), np.mean(valid_y))

        clf = LGBMClassifier(
            nthread=10,
            n_estimators=30000,
            learning_rate=lr,
            num_leaves=num_leaves,
            colsample_bytree=colsample, # 0.67
            subsample=subsample,
            subsample_freq=0, ## disable subsampling
            max_depth=max_depth,
#             reg_alpha=0.65,
            reg_lambda=reg_lambda,
#             min_split_gain=0.0222415,
#             min_child_weight=39.3259775,
            min_child_samples=min_child_samples,
            silent=-1,
            verbose=-1, )
        if rounds is not None:
            clf.n_estimators = rounds
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric='logloss', verbose=verbose)
            oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats])[:, 1] 
        else:
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose, early_stopping_rounds= 200)
            oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats], num_iteration=clf.best_iteration_)[:, 1] 
            
        holdout_final_preds += holdout_preds / folds.n_splits
        score = log_loss(valid_y, oof_preds[valid_idx])
        train_score = clf.best_score_['training']['binary_logloss']
        holdout_score = log_loss(holdout['click'], holdout_preds)
        diff = abs(score - holdout_score)
        best_rounds = rounds if rounds is not None else clf.best_iteration_
        print('Fold %2d [%5d] AUC : ho: %.6f / te: %.6f / tr: %.6f (diff: %.6f)' % (n_fold + 1, best_rounds, holdout_score, score,  train_score, diff))
        scores.append(score)
        train_scores.append(train_score)
        holdout_scores.append(holdout_score)
        diff_val_holdout.append(diff)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        fold_importance_gain_df = pd.DataFrame()
        fold_importance_gain_df["feature"] = feats
        fold_importance_gain_df["importance"] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance_gain_df["fold"] = n_fold + 1
        feature_importance_gain_df = pd.concat([feature_importance_gain_df, fold_importance_gain_df], axis=0)
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    holdout_roc = roc_auc_score(holdout['click'], holdout_final_preds)
    holdout_mean = np.mean(holdout_scores)
    full_te_mean = np.mean(scores)
    full_tr_mean = np.mean(train_scores)
#     print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    print('Full HO score %.6f' % holdout_roc)
    print('FULL HO mean {:.6f}, std {:.6f}'.format(holdout_mean, np.std(holdout_scores)))
    print('FULL TE mean {:.6f}, std {:.6f}'.format(full_te_mean, np.std(scores)))
    print('FULL TR mean {:.6f}, std {:.6f}'.format(full_tr_mean, np.std(train_scores)))
    print('FULL DIFF mean {:.6f}, std {:.6f}'.format(np.mean(diff_val_holdout), np.std(diff_val_holdout)))
    # Write submission file and plot feature importance
    if not debug:
        test_df['predicted_score'] = sub_preds
        test_df[['instance_id', 'predicted_score']].to_csv(submission_file_name, index= False)
#     if not print_to_file:
#         display_importances(feature_importance_df, fe_img_name)
    feature_importance_df = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    feature_importance_gain_df = feature_importance_gain_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    return feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds 

# Display/plot feature importance
def display_importances(feature_importance_df_, fe_img_name):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig(fe_img_name+'.png')


def convert_and_save_imp_df(fe_imp_df, dumpfilename):
    fe_imp_df_mean = fe_imp_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    pickle.dump(fe_imp_df_mean, open(dumpfilename,'wb'))

In [28]:
def runlgb(train, test, holdout):
    colsamples = [0.07]#[0.1,0.15,0.2]#[0.03,0.04,0.05,0.06,0.07,0.08]
    seeds = [20]#[300,4000,50000,600000,7000000,80000000,523445,31275479] # 20
    depth = [5]
    leaves = [16]
    min_child_sam = [20]#, 800]
    subsamples = [1]#0.8, 0.7, 0.6, 0.5, 0.4] # was 1
    reg_lambdas = [0.5]
    # lrs = lrs.tolist()
    lrs2 = [0.05]
    nfolds = 5
    rounds = [None] #[1000]#, 1300, 1600, 1900, 2200, 2500]
    for seed in seeds:
        for colsample in colsamples:
            for d in depth:
                for l in leaves:
                    for mcs in min_child_sam:
                        for subsample in subsamples:
                            for reg_lambda in reg_lambdas:
                                for lr in lrs2:
                                    for r in rounds:
                                        filename = 'fe_936_col{}_lr{}_n{}'.format(len(train.columns), lr, nfolds)
                                        print('#############################################')
                                        print(colsample, seed, d, l, mcs, subsample, reg_lambda, lr, 'nfolds:', nfolds)
                                        print('#############################################')
                                        numfeats = len(train.columns)
                                        with timer("Run LightGBM with kfold"):
                                            return kfold_lightgbm(None, train, test, holdout, nfolds, filename+'.csv', filename, colsample=colsample, verbose=100, max_depth=d, num_leaves=l, min_child_samples=mcs, subsample=subsample, reg_lambda=reg_lambda, lr=lr, seed=seed, stratified=True, rounds=r,debug=False)                                        

In [5]:
train_df, holdout = train_test_split(train, test_size=1/10000, random_state=42)
#42
print('MEAN: train({}) vs holdout({}): '.format(len(train_df), len(holdout)), train_df['click'].mean(), holdout['click'].mean())
print(train_df.shape, test.shape, holdout.shape)

MEAN: train(1001549) vs holdout(101):  0.19846158300792074 0.1782178217821782
(1001549, 809) (40024, 808) (101, 809)


In [6]:
feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train, test, holdout)

#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(1001650, 809) (40024, 808) (101, 809)
MEAN: train(1001650) vs holdout(101):  0.19845954175610242 0.1782178217821782
Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.419999	valid_1's binary_logloss: 0.420441
[200]	training's binary_logloss: 0.416112	valid_1's binary_logloss: 0.417871
[300]	training's binary_logloss: 0.413917	valid_1's binary_logloss: 0.417052
[400]	training's binary_logloss: 0.41237	valid_1's binary_logloss: 0.41675
[500]	training's binary_logloss: 0.411016	valid_1's binary_logloss: 0.416597
[600]	training's binary_logloss: 0.409863	valid_1's binary_logloss: 0.416547
[700]	training's binary_logloss: 0.40866	valid_1's binary_logloss: 0.416485
[800]	training's binary_logloss: 0.40747	valid_1's binary_logloss: 0.416429
[900]	training's binary_logloss: 0.406333	valid_1's binary_logloss: 0.416383
[1

In [6]:
feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train, test, holdout)

#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(1001650, 1668) (40024, 1667) (101, 1668)
MEAN: train(1001650) vs holdout(101):  0.19845954175610242 0.1782178217821782
Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.419158	valid_1's binary_logloss: 0.419554
[200]	training's binary_logloss: 0.415661	valid_1's binary_logloss: 0.41747
[300]	training's binary_logloss: 0.413587	valid_1's binary_logloss: 0.416878
[400]	training's binary_logloss: 0.4119	valid_1's binary_logloss: 0.416654
[500]	training's binary_logloss: 0.410438	valid_1's binary_logloss: 0.41655
[600]	training's binary_logloss: 0.40917	valid_1's binary_logloss: 0.416511
[700]	training's binary_logloss: 0.407961	valid_1's binary_logloss: 0.416508
[800]	training's binary_logloss: 0.406799	valid_1's binary_logloss: 0.416514
Early stopping, best iteration is:
[670]	training's binary_logloss: 0.408342	

# Select features

In [7]:
# base_gain = feature_importance_gain_df.copy()
# base_split = feature_importance_df.copy()

In [34]:
gain_fe = base_gain.iloc[:800]['feature'].values
split_fe = base_split.iloc[:800]['feature'].values
all_fe = set(gain_fe).intersection(set(split_fe))
print(len(all_fe))

797


In [35]:
all_fe = list(all_fe)
all_fe.append('instance_id')
train_fe = list(all_fe).copy()
train_fe.append('click')

train_new = train[train_fe].copy()
test_new = test[all_fe].copy()

In [36]:
train_df, holdout = train_test_split(train_new, test_size=1/10000, random_state=42)
#42
print('MEAN: train({}) vs holdout({}): '.format(len(train_df), len(holdout)), train_df['click'].mean(), holdout['click'].mean())
print(train_df.shape, test.shape, holdout.shape)

MEAN: train(1001549) vs holdout(101):  0.19846158300792074 0.1782178217821782
(1001549, 799) (40024, 809) (101, 799)


In [37]:
df_split, df_gain,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train_new, test_new, holdout)

#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(1001650, 799) (40024, 798) (101, 799)
MEAN: train(1001650) vs holdout(101):  0.19845954175610242 0.1782178217821782
Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.420173	valid_1's binary_logloss: 0.420356
[200]	training's binary_logloss: 0.416333	valid_1's binary_logloss: 0.417832
[300]	training's binary_logloss: 0.414112	valid_1's binary_logloss: 0.417094
[400]	training's binary_logloss: 0.41237	valid_1's binary_logloss: 0.416684
[500]	training's binary_logloss: 0.411016	valid_1's binary_logloss: 0.416611
[600]	training's binary_logloss: 0.409883	valid_1's binary_logloss: 0.416565
[700]	training's binary_logloss: 0.408637	valid_1's binary_logloss: 0.4165
[800]	training's binary_logloss: 0.407571	valid_1's binary_logloss: 0.416513
[900]	training's binary_logloss: 0.406516	valid_1's binary_logloss: 0.416513
E

In [11]:
train_new.to_pickle(FILE.train_agg_v1_select.value)
test_new.to_pickle(FILE.test_agg_v1_select.value)

In [41]:
base_split.to_csv('../../data/features/agg_v1_select_feature_importance_split.csv',index=False)
base_gain.to_csv('../../data/features/agg_v1_select_feature_importance_gain.csv',index=False)